In [9]:
# First cell - imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import requests
from datetime import datetime
import json
import os
from dotenv import load_dotenv

In [10]:
load_dotenv()

# Get API key from environment
API_KEY = os.getenv('FRED_API_KEY')
BASE_URL = 'https://api.stlouisfed.org/fred'

def get_fred_series(series_id, observation_start=None):
    """
    Fetch data series from FRED API
    
    Parameters:
    series_id (str): FRED series identifier
    observation_start (str): Start date in YYYY-MM-DD format
    
    Returns:
    pandas.DataFrame: Time series data
    """
    # Build API URL
    url = f"{BASE_URL}/series/observations"
    
    params = {
        'series_id': series_id,
        'api_key': API_KEY,
        'file_type': 'json',
        'observation_start': observation_start if observation_start else '1976-01-01'
    }
    
    # Make API request
    response = requests.get(url, params=params)
    data = response.json()
    
    # Convert to DataFrame
    df = pd.DataFrame(data['observations'])
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    return df.set_index('date')['value']

# Get our series
yield_spread = get_fred_series('T10Y2Y')
gdp = get_fred_series('GDPC1')
fed_funds = get_fred_series('DFF')
unenmployment = get_fred_series('UNRATE')

# Create main dataframe
df = pd.DataFrame({
    'yield_spread': yield_spread,
    'gdp': gdp,
    'fed_funds':fed_funds, 
    'unemployment': unenmployment
})

# Calculate GDP growth rate (annualized)
df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100

#export to excel to validate 
df.to_excel('stats.xlsx')

C:\Users\mrasm\AppData\Local\Temp\ipykernel_28256\4152973758.py:54: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100


In [11]:
# Show first few rows
display(df.head())

# Show basic statistics
display(df.describe())

,yield_spread,gdp,fed_funds,unemployment,gdp_growth
date,,,,,
1976-01-01,NaN,6323.649,5.37,7.9,NaN
1976-01-02,NaN,NaN,5.28,NaN,NaN
1976-01-03,NaN,NaN,5.28,NaN,NaN
1976-01-04,NaN,NaN,5.28,NaN,NaN
1976-01-05,NaN,NaN,5.29,NaN,0.0


,yield_spread,gdp,fed_funds,unemployment,gdp_growth
count,12109.000000,195.000000,17844.000000,586.000000,17841.000000
mean,0.859588,13757.474856,4.669866,6.116382,0.029676
std,0.926131,4903.223297,3.968819,1.757661,0.264554
min,-2.410000,6323.649000,0.040000,3.400000,-7.908965
25%,0.180000,9429.048000,1.160000,4.725000,0.000000
50%,0.820000,14130.908000,4.790000,5.800000,0.000000
75%,1.500000,17457.087500,6.590000,7.200000,0.000000
max,2.910000,23386.248000,22.360000,14.800000,7.830225
